# Building Deep Learning Applications with Keras: Hello Keras!

The best way to describe Keras is by its [official site](https://keras.io/)

> Keras is a high-level neural networks API, written in Python and capable of running on top of either TensorFlow or Theano. It was developed with a focus on enabling fast experimentation. Being able to go from idea to result with the least possible delay is key to doing good research."

**In this tutorial we will will learn a very basic usage of this library.**


In [ ]:
import keras
import numpy as np
from util import get_housing_prices_data, randomize_in_place, standardize, r_squared
from plots import plot_points_regression 

## Linear Regression example

As a starting point we will use a synthetic data of housing prices. Here we have observations of the form $(x_1, y_1) \dots (x_N, y_N)$ where $x_i$ is size (in square-meters) and $y_i$ is price. The learning task here is simple: **use past observations to predict prices**.

## House Prices: a toy example

First, let's generate two types of data: the **train data** (which we use to train our model) and the **test data** (a part of the data used only to assess the performance of the model).

In [ ]:
X, y = get_housing_prices_data(N=300, verbose=False)
randomize_in_place(X, y)

train_X = X[0:250]
train_y = y[0:250]
test_X = X[250:]
test_y = y[250:]

We can use the library [matplotlib](https://matplotlib.org/) to get useful visualizations of the data

In [ ]:
plot_points_regression(train_X,
                       train_y,
                       title='Real estate prices prediction (train data)',
                       xlabel="m\u00b2",
                       ylabel='$')

In [ ]:
plot_points_regression(test_X,
                       test_y,
                       title='Real estate prices prediction (test data)',
                       xlabel="m\u00b2",
                       ylabel='$')

## The model

We will start with the basic model in the machine learning toolkit: **the linear model**. Here a linear model $f$ will use only two parameters $w$ and $b$ to estimate the price:

$$
\hat{y}_{i} = f(x_{i}; w, b) = wx_{i} + b
$$

To estimate the quality of the model we use one metric to check how far away is the model's output from the target values. In this case, we use [the mean squared error](https://en.wikipedia.org/wiki/Mean_squared_error): 

\begin{equation}
J(w, b) = \frac{1}{N}\sum_{i=1}^{N}(\hat{y}_{i} - y_{i})^{2}
\end{equation}

It's very easy to define this model using Keras.

An easy way to define models in Keras is by using the [`Sequential`](https://keras.io/getting-started/sequential-model-guide/) class. This class creates a model when we pass to it a sequence of layers.

In [ ]:
from keras import Sequential
from keras.layers import Dense

We have only one feature ($m^2$) and only one output (the price), so we can see a linear regression model as a very simplistic neural network with no hidden layer and with only one neuron.

In [ ]:
linear_regression = keras.Sequential([Dense(units=1, input_shape=[1])])

We now compile the model creating the **loss function** and the **optimizer**.

In [ ]:
linear_regression.compile(loss='mean_squared_error', 
              optimizer='sgd')

There are different loss functions and optimizers available in Keras:

**Loss functions**

- categorical_crossentropy
- sparse_categorical_crossentropy
- binary_crossentropy
- mean_squared_error
- mean_absolute_error

**Optimizers**
- SGD
- RMSprop
- Adagrad
- Adadelta
- Adam
- Adamax

Now to perform the training let's stantardize the data

In [ ]:
train_X_norm = standardize(train_X)
train_y_norm = standardize(train_y)
test_X_norm = standardize(test_X)

A nontrainded model starts with a random prediction. We always can use the model to predict by using the `.predict()` method.

In [ ]:
pred = linear_regression.predict(train_X_norm)
pred = pred.reshape((pred.shape[0],))
plot_points_regression(train_X_norm,
                       train_y_norm,
                       prediction=pred,
                       title='Real estate prices prediction (test set)',
                       xlabel="m\u00b2",
                       ylabel='$')

To train the model we use the `.fit()` method.

In [ ]:
linear_regression.fit(train_X_norm,
                      train_y_norm,
                      batch_size=32,
                      epochs=20,
                      validation_split=0.2,
                      verbose=1)

After training, we can see the model's prediction quality 

In [ ]:
pred = linear_regression.predict(train_X_norm)
pred = pred.reshape((pred.shape[0],))
plot_points_regression(train_X_norm,
                       train_y_norm,
                       prediction=pred,
                       title='Real estate prices prediction (test set)',
                       xlabel="m\u00b2",
                       ylabel='$')

The model have fitted the trained data, great. But remember, **that is not what we want!** We want to predict unseen data. Let's see how the model behave on the **test data**. We can also use the [$R^2$](https://en.wikipedia.org/wiki/Coefficient_of_determination) metric do see how well the linear model fits the test data.

In [ ]:
prediction = linear_regression.predict(test_X_norm)
prediction = (prediction * np.std(train_y)) + np.mean(train_y)
r_2 = r_squared(test_y, prediction)

plot_points_regression(test_X,
                       test_y,
                       title='Test data',
                       xlabel="m\u00b2",
                       ylabel='$',
                       prediction=prediction,
                       r_squared=r_2,
                       legend=True)

## House Prices: Kaggle example

You have learned the basics of linear regression here. But this data set is just a toy example. You can find a more realistic data of the same problem in [Kaggle](https://www.kaggle.com/c/house-prices-advanced-regression-techniques). Check it out!
